In [9]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [10]:
MAIN_FOLDER = main_folder("undergroundmarket")

create_database('blp_dataset')
create_table('products_tmp')

In [12]:
folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]
original_images = []
original_names = []

import base64

for fo in folders:
    folder = Path(f"{fo}\item")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for file in os.listdir(folder):
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())
            page_content = page_content.find("div", {"class": "itemInfo"})

            if page_content is not None:
                ## GET DESCRIPTION
                description = ""
                desc_content = page_content.find("div", {"id": "main"})
                
                if desc_content is not None:
                    description = get_content(desc_content)
                    
                ## GET TITLE
                title = ""
                raw_title = page_content.select(".itemInfo > h2:nth-of-type(2)")
                
                if raw_title is not None and len(raw_title) > 0:
                    title = get_content(raw_title[0])

                ## GET SELLER
                seller = ""
                seller_content = page_content.find("p", {"class": "vendor"}).text

                if seller_content is not None:
                    seller_raw = get_content(seller_content)
                    seller = re.sub('[(10-9)]', '', seller_raw)


                ## GET PRICE
                price = ""
                price_content = page_content.find("div", {"class": "price"}).text
                
                if price_content is not None:
                    price_raw = get_content(price_content)
                    price = re.sub('[Price: ]', '', price_raw)                    
                    

                if description != "" or title != "":
                    # Insert product into database
                    query = "INSERT INTO products (name, description, market_name, seller_name, price, illegal) VALUES (%s, %s, %s, %s, %s, %s)"
                    values = (title, description, "Underground Market", seller, price, "t")

                    cursor.execute(query, values)
                    conn.commit()

                    product_id = cursor.lastrowid
                    
                    ## GET IMAGE
                    images_content = page_content.find_all("img")

                    if images_content is not None:
                        images_content = [img['src'] for img in images_content]

                        cont = 1

                        for image in images_content:
                            img_name = product_id

                            if len(images_content) > 1:
                                c = f"{product_id}_{cont}"
                                cont = cont + 1
                            else:
                                if image in original_images:
                                    query = "UPDATE products SET has_image = %s WHERE id = %s"
                                    cursor.execute(query, ("no_image", product_id))
                                    conn.commit()

                            if image not in original_images:
                                original_images.append(image)
                                original_names.append(img_name)
                                dst_dir = Path(f"D:\images\pandora\{product_id}.jpg")

                                # save image
                                try:
                                    with open(dst_dir, "wb") as fh:
                                        fh.write(base64.b64decode(image))
                                except:
                                    query = "UPDATE products SET has_image = %s WHERE id = %s"
                                    cursor.execute(query, ("no_image", product_id))
                                    conn.commit()
                    else:
                        query = "UPDATE products SET has_image = %s WHERE id = %s"
                        cursor.execute(query, ("no_image", product_id))
                        conn.commit()

In [121]:
num = num + 1


print(original_names[num])
print(original_images[num])

IndexError: list index out of range